# Tutorial: Orchestrate Function Calling with Mistralai/Mistral-7B-Instruct-v0.1 using ActionWeaver through Anyscale Endpoints

Welcome to this tutorial where we will guide you through using ActionWeaver to orchestrate function calling with the open-source model mistralai/Mistral-7B-Instruct-v0.1 via Anyscale endpoints.

## Prerequisites

Before we start, ensure you have the following:

- An Anyscale account
- API key for Anyscale

Follow the instructions to set up your account and obtain your API key: [Anyscale Documentation](https://docs.endpoints.anyscale.com/?_gl=1*1t5jz2h*_ga*ODk0Nzg3NTM1LjE3MDE5ODg4NzM.*_ga_T6EXHYG44V*MTcwMjc2NDYyNi43LjEuMTcwMjc2NDY2Ny4xOS4wLjA)

Happy coding!

## Additional Resources

For more details regarding ActionWeaver please visit the [ActionWeaver GitHub repository](https://github.com/TengHu/ActionWeaver).

Explore the repository to understand more about its capabilities, access documentation, and view the cookbook to get started!


**Step 1**: Patch OpenAI client

In [72]:

from actionweaver.llms import patch
from openai import OpenAI
import os

openai_client = patch(OpenAI(
    base_url = "https://api.endpoints.anyscale.com/v1",
    api_key = os.getenv("ANYSCALE_API_TOKEN"),
))

**Step 2**: Define function you want model to invoke

In [54]:
from actionweaver import action

@action(name="GetCurrentTime")
def get_current_time() -> str:
    """
    Use this for getting the current time in the specified time zone.
    
    :return: A string representing the current time in the specified time zone.
    """
    print ("Getting current time...")
    import datetime
    current_time = datetime.datetime.now()
    
    return f"The current time is {current_time}"



**Step 3**: invoke the chat completion api with mistralai/Mistral-7B-Instruct-v0.1

In [86]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "what time is it now"}
  ]


response = openai_client.chat.completions.create(
  model="mistralai/Mistral-7B-Instruct-v0.1",
  messages=messages,
  stream=False,
  temperature=0.7,
  actions=[get_current_time],
)

response

Getting current time...


ChatCompletion(id='mistralai/Mistral-7B-Instruct-v0.1-1r6q1D_zIMOZlpQBCZsHC_pezU96tAjlRM2iEvPiG7E', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=' The current time is 2023-12-28 16:23:40.227544.', role='assistant', function_call=None, tool_calls=None, tool_call_id=None))], created=1703798621, model='mistralai/Mistral-7B-Instruct-v0.1', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=42, prompt_tokens=638, total_tokens=680))

In [75]:
get_current_time.invoke(openai_client, messages=[{"role": "user", "content": "what time"}], model="mistralai/Mistral-7B-Instruct-v0.1", stream=False, force=False)

Getting current time...


ChatCompletion(id='mistralai/Mistral-7B-Instruct-v0.1-o2VxZXY00gFNlgN-Jjdy04C1b2h20c11_X_kMwjE0sA', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=' The answer to the question "What is the current time?" is "2023-12-28 16:14:14.601829".', role='assistant', function_call=None, tool_calls=None, tool_call_id=None))], created=1703798055, model='mistralai/Mistral-7B-Instruct-v0.1', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=51, prompt_tokens=660, total_tokens=711))